In [1]:
#Eighty percent of our business comes from 20% of our customers.
#It costs 10 times less to sell to an existing customer than to find a new customer

In [ ]:
import numpy as np
import pandas as pd
import time, warnings
import datetime as dt
warnings.filterwarnings("ignore")

df=pd.read_csv("retail_data.csv")
df.head()

# RFM Analysis
## Recency
# To calculate recency, we need to find out  **when was the customer's most recent purchase.**.
# Create a new column called date which contains the date of invoice only

df['date'] = pd.DatetimeIndex(df['InvoiceDate']).date
# Group by customers and check last date of purchase
recency_df = df.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurshaceDate']
# Calculate recent date to find recency wrt to this date
recent_date=recency_df.LastPurshaceDate.max()
print(recent_date)
# Calculate recency
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (recent_date - x).days)
recency_df.head()

In [ ]:
# ## Frequency
# To calculate Frequency we need to check **How often a customer makes a purchase**.
# Drop duplicates
df1= df
df1.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)
# Calculate the frequency of purchases
frequency_df = df1.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

In [ ]:
# ## Monetary
# To calculate Monetary value  **How much money did the customer spent during the timeframe?**
# Create column total cost
df['TotalCost'] = df['Quantity'] * df['UnitPrice']
monetary_df = df.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

In [ ]:
# ## Create RFM Table
# Merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(frequency_df,on='CustomerID')
temp_df.head()
# Merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='CustomerID')
# Use CustomerID as index
rfm_df.set_index('CustomerID',inplace=True)
# Check the head
rfm_df.head()

In [ ]:
# Rank each metric R , F & M
rfm_df['R_rank'] = rfm_df['Recency'].rank( ascending=False)
rfm_df['F_rank'] = rfm_df['Frequency'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['Monetary'].rank(ascending=True)
rfm_df.head()
# normalize each rank with Max rank
rfm_df['R_rank_norm']=(rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm']=(rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm']=(rfm_df['F_rank']/rfm_df['M_rank'].max())*100
rfm_df.head()
# Now apply our equation and create final score **Consolidated Score = 0.15*R_rank_norm + 0.28*F_rank_norm + 0.57M_rank_norm**
rfm_df['RFM_Score']=0.15*rfm_df['R_rank_norm']+0.28*rfm_df['F_rank_norm']+0.57*rfm_df['M_rank_norm']
rfm_df=rfm_df.round(0)
rfm_df.head()

In [ ]:
# ## Customer segments with RFM Model
# # Segment customers based on RFM score
# 0 - 50 - Low valued customer
# 50 - 75 - Medium valued customer
# 76 - 100 - High valued customer
rfm_df["Customer_segment"]=np.where(rfm_df['RFM_Score'] > 75 ,"High Value Customer",(np.where(rfm_df['RFM_Score'] < 50 , "Low value Customer" ,"Medium Value Customer")))
rfm_df.head()

In [ ]:
#High Value Customer: They are your loyal customers.So provide constant support via customer service
#Medium Value Customer: Send them personalized emails with offers and encourage them to shop more
#Low Value Customer: These customers are about to churn or went into a dormant stage,apply reactivation strategies for them